In [1]:
import sys
import os
# Add the root directory of the project to PYTHONPATH
sys.path.append(os.path.abspath(os.path.join('../../master-thesis-dizio-ay2324')))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from fairlib import DataFrame
from fairlib.preprocessing import DisparateImpactRemover
from fairlib.metrics import statistical_parity_difference, disparate_impact
from fairlib.preprocessing.lfr import LFR
random_state = 42
np.random.seed(random_state)

/home/lyudmil/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [3]:
import pandas as pd

df_cleaned = pd.read_csv('cleaned_dataset.csv')

In [4]:
features = [
    # Base Attributes
    "Sex_int",
    "Protected category",
    "Overall",
    "Technical Skills",
    "Standing/Position",
    "Comunication",
    "Maturity",
    "Dynamism",
    "Mobility",
    "English",
    "Italian Residence",
    "European Residence",
    "Age Range_int",
    "number_of_searches",
    # Custom Similarity Scores
    "experience_match_score",
    "current_salary_fit_score",
    "expected_salary_fit_score",
    "study_title_score",
    "professional_similarity_score",
    "study_area_score",
    "general_similarity_score",
    "Distance Residence - Akkodis HQ",
    "Distance Residence - Assumption HQ",
]

In [5]:
protected_attributes = [
    'Sex_int', 'Protected category', 'Age Range_int',
    'Italian Residence', 'European Residence'
]

In [6]:
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score
)
from fairlearn.metrics import (
    demographic_parity_ratio,
    equalized_odds_ratio
)
from sklearn.impute import SimpleImputer
from fairlib.inprocessing import AdversarialDebiasing
from tqdm import tqdm
import numpy as np
from imblearn.over_sampling import SMOTE
# Preprocessing
dataset = DataFrame(df_cleaned[features + ['Hired']])
bool_cols = dataset.select_dtypes(include='bool').columns
non_bool_cols = dataset[features].columns.difference(bool_cols)
dataset[bool_cols] = dataset[bool_cols].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    dataset[features], dataset['Hired'], test_size=0.2
)

imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# Normalize non-binary features
scaler = StandardScaler()
X_train[non_bool_cols] = scaler.fit_transform(X_train[non_bool_cols])
X_test[non_bool_cols] = scaler.transform(X_test[non_bool_cols])

smote = SMOTE(sampling_strategy=0.75, random_state=42)  # 0.75 means minority will be 75% of majority
X_train, y_train = smote.fit_resample(X_train, y_train)

# Adversarial Debiasing with Multiple Fairness Metrics
for sensitive_col in tqdm(protected_attributes):
    print(f'\nAttribute: {sensitive_col}')
    for lambda_adv in [0, 0.5, 1]:
        print(f'\nLambda Adv: {lambda_adv}')
        
        X_train_dataframe = DataFrame(X_train)
        X_train_dataframe.sensitive = sensitive_col

        baseline_model = AdversarialDebiasing(
            input_dim=X_train.shape[1],
            hidden_dim=8,
            output_dim=1,
            sensitive_dim=1,
            lambda_adv=lambda_adv,
        )
        # Fit the model
        baseline_model.fit(X_train_dataframe, y_train)

        X_test_dataframe = DataFrame(X_test)
        X_test_dataframe.sensitive = sensitive_col

        y_pred_tensor = baseline_model.predict(X_test_dataframe)

        y_pred_baseline = y_pred_tensor.detach().cpu().numpy()

        # Standard performance metrics
        accuracy = accuracy_score(y_test, y_pred_baseline)
        precision = precision_score(y_test, y_pred_baseline, zero_division=0)
        recall = recall_score(y_test, y_pred_baseline, zero_division=0)
        f1 = f1_score(y_test, y_pred_baseline, zero_division=0)
        roc_auc = roc_auc_score(y_test, y_pred_baseline)

        # Fairness metrics
        spd = statistical_parity_difference(y_pred_baseline, X_test_dataframe[sensitive_col])
        di = disparate_impact(y_pred_baseline, X_test_dataframe[sensitive_col])
        dpr = demographic_parity_ratio(y_test, y_pred_baseline, sensitive_features=X_test_dataframe[sensitive_col])
        eor = equalized_odds_ratio(y_test, y_pred_baseline, sensitive_features=X_test_dataframe[sensitive_col])

        # Print results
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"ROC AUC: {roc_auc:.4f}")
        print(f"Statistical Parity Difference (SPD): {spd}")
        print(f"Disparate Impact (DI): {di}")
        print(f"Demographic Parity Ratio: {dpr}")
        print(f"Equalized Odds Ratio: {eor}")


  0%|          | 0/5 [00:00<?, ?it/s]


Attribute: Sex_int

Lambda Adv: 0


  0%|          | 0/5 [01:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument weight in method wrapper_CUDA__native_batch_norm)

In [8]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

dataset = DataFrame(df_cleaned[features+['Hired']])

bool_cols = dataset.select_dtypes(include='bool').columns
non_bool_cols = dataset[features].columns.difference(bool_cols)


dataset[bool_cols] = dataset[bool_cols].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    dataset[features], dataset['Hired'], test_size=0.2
)
scaler = StandardScaler()
X_train[non_bool_cols] = scaler.fit_transform(X_train[non_bool_cols])
X_test[non_bool_cols] = scaler.transform(X_test[non_bool_cols])



from fairlib.inprocessing import AdversarialDebiasing
from tqdm import tqdm
for sensitive_col in tqdm(protected_attributes):
    print(f'Attribute: {sensitive_col}')
    for lambda_adv in [0, 0.5, 1]:
        print(f'Lambda Adv: {lambda_adv}')
        X_train_dataframe = DataFrame(X_train)
        X_train_dataframe.sensitive = sensitive_col

        baseline_model = AdversarialDebiasing(
            input_dim=X_train.shape[1],
            hidden_dim=8,
            output_dim=1,
            sensitive_dim=1,
            lambda_adv=lambda_adv,
        )
        # Fit the model
        baseline_model.fit(X_train_dataframe, y_train)

        X_test_dataframe = DataFrame(X_test)
        X_test_dataframe.sensitive = sensitive_col

        y_pred_tensor = baseline_model.predict(X_test_dataframe)

        y_pred_baseline = y_pred_tensor.detach().cpu().numpy()

        baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
        
        baseline_spd = statistical_parity_difference(y_pred_baseline, X_test_dataframe[sensitive_col])
        baseline_di = disparate_impact(y_pred_baseline, X_test_dataframe[sensitive_col])

        # Print the fairness metrics
        print(f"Baseline model accuracy: {baseline_accuracy:.4f}")
        print(f"Statistical Parity Difference (SPD): {baseline_spd}")
        print(f"Disparate Impact (DI): {baseline_di}")

  0%|          | 0/5 [00:00<?, ?it/s]

Attribute: Sex_int
Lambda Adv: 0


/home/lyudmil/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Baseline model accuracy: 0.8994
Baseline model accuracy: 0.8994350282485876
Statistical Parity Difference (SPD): [[0.]
 [0.]]
Disparate Impact (DI): [[1.]
 [1.]]
Lambda Adv: 0.5
Baseline model accuracy: 0.8994
Baseline model accuracy: 0.8994350282485876
Statistical Parity Difference (SPD): [[0.]
 [0.]]
Disparate Impact (DI): [[1.]
 [1.]]
Lambda Adv: 1


 20%|██        | 1/5 [03:11<12:44, 191.04s/it]

Baseline model accuracy: 0.8994
Baseline model accuracy: 0.8994350282485876
Statistical Parity Difference (SPD): [[0.]
 [0.]]
Disparate Impact (DI): [[1.]
 [1.]]
Attribute: Protected category
Lambda Adv: 0
Baseline model accuracy: 0.8994
Baseline model accuracy: 0.8994350282485876
Statistical Parity Difference (SPD): [[0.]
 [0.]]
Disparate Impact (DI): [[1.]
 [1.]]
Lambda Adv: 0.5
Baseline model accuracy: 0.8994
Baseline model accuracy: 0.8994350282485876
Statistical Parity Difference (SPD): [[0.]
 [0.]]
Disparate Impact (DI): [[1.]
 [1.]]
Lambda Adv: 1


 40%|████      | 2/5 [06:56<10:34, 211.38s/it]

Baseline model accuracy: 0.8994
Baseline model accuracy: 0.8994350282485876
Statistical Parity Difference (SPD): [[0.]
 [0.]]
Disparate Impact (DI): [[1.]
 [1.]]
Attribute: Age Range_int
Lambda Adv: 0
Baseline model accuracy: 0.8994
Baseline model accuracy: 0.8994350282485876
Statistical Parity Difference (SPD): [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Disparate Impact (DI): [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
Lambda Adv: 0.5
Baseline model accuracy: 0.8994
Baseline model accuracy: 0.8994350282485876
Statistical Parity Difference (SPD): [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Disparate Impact (DI): [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
Lambda Adv: 1


 40%|████      | 2/5 [11:01<16:32, 330.70s/it]


KeyboardInterrupt: 

In [ ]:

baseline_model = AdversarialDebiasing(
    input_dim=X_train.shape[1],
    hidden_dim=8,
    output_dim=1,
    sensitive_dim=1,
    lambda_adv=0.5,
)
# Fit the model
baseline_model.fit(X_train, y_train)

X_test_dataframe = DataFrame(X_test)
X_test_dataframe.sensitive = SENSITIVE_COL_NAME

y_pred_tensor = baseline_model.predict(X_test)

y_pred_baseline = y_pred_tensor.detach().cpu().numpy()

baseline_accuracy = accuracy_score(y_test, y_pred_baseline)
print(f"Baseline model accuracy: {baseline_accuracy:.4f}")

baseline_spd = statistical_parity_difference(y_pred_baseline, X_test_dataframe[SENSITIVE_COL_NAME])
baseline_di = disparate_impact(y_pred_baseline, X_test_dataframe[SENSITIVE_COL_NAME])

# Print the fairness metrics
print(f"Baseline model accuracy: {baseline_accuracy}")
print(f"Statistical Parity Difference (SPD): {baseline_spd}")
print(f"Disparate Impact (DI): {baseline_di}")